- 为了对比原始DQN算法与9,10两节算法，在相同Pendulum-v1环境与超参数下，不同网络结构性能的差异,特此在本节展示原始DQN算法的性能

#### 倒立摆（Inverted Pendulum）环境：
> 倒摆摆动问题基于控制理论中的经典问题。 该系统由一端连接到固定点的摆锤组成，另一端是自由的:
![pendulum环境示意图](Illustrations/pendulum环境.gif)

> 钟摆从一个随机位置开始，起始状态是 [-pi， pi] 中的随机角度和 [-1,1] 中的随机角速度
目标是在自由端施加扭矩以摆动它进入直立位置，在 200 个时间步长处截断
$\theta$以弧度为单位，弧度 = 角度 × $\frac{\pi}{180}$:
![pendulum坐标系](Illustrations/Pendulum坐标系.png)

> 环境的状态包括倒立摆角度的正弦值$sin\theta$，余弦值$cos\theta$，角速度$\dot{\theta}$；动作为对倒立摆施加的力矩torque:
![pendulum环境的状态空间](Illustrations/Pendulum环境的状态空间.png)
![pendulum环境的动作空间](Illustrations/Pendulum环境的动作空间.png)

> 奖励函数定义为：
$$r=-(theta^2+0.1*theta_-dt^2+0.001*torque^2)$$
可见，单次获得的最低奖励为 $(\text{圆周率}^2+0.1*8^2+0.001*2^2)=-16.2736044$, 而最大奖励为 0（钟摆直立，角速度为零，不施加力矩）

导入相关库：

In [2]:
import random
import numpy as np
from tqdm import tqdm
import torch
import torch.nn.functional as F
import gymnasium as gym

In [ ]:
from utils.replay_buffer import ReplayBuffer
from utils.smoothing import moving_average

#### Q网络

In [3]:
class Qnet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(Qnet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)